In [1]:
import pandas as pd

In [6]:
df = pd.read_csv("data/satellite_data.csv")
df.head()

,OBJECT_NAME,OBJECT_ID,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT
0,CALSPHERE 1,1964-063C,2024-10-04T21:04:06.723840,13.753923,0.002531,90.2077,57.7181,13.2889,140.5397,0,U,900,999,98610,0.003792,3.659000e-05,0.0
1,CALSPHERE 2,1964-063E,2024-10-04T21:20:37.934880,13.528209,0.001716,90.2215,61.4004,223.8250,307.5836,0,U,902,999,77239,0.000264,1.840000e-06,0.0
2,LCS 1,1965-034C,2024-10-04T20:44:33.029952,9.893067,0.000814,32.1379,209.8915,78.6346,281.5066,0,U,1361,999,14845,0.002065,2.400000e-07,0.0
3,TEMPSAT 1,1965-065E,2024-10-04T20:36:45.512640,13.335324,0.006918,89.9646,213.6337,189.0694,232.1401,0,U,1512,999,87737,0.000278,1.480000e-06,0.0
4,CALSPHERE 4A,1965-065H,2024-10-04T20:04:34.135680,13.361109,0.007081,89.9358,127.4565,54.7174,75.9291,0,U,1520,999,87986,0.000580,3.110000e-06,0.0


In [7]:
df.columns

Index(['OBJECT_NAME', 'OBJECT_ID', 'EPOCH', 'MEAN_MOTION', 'ECCENTRICITY',
       'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY',
       'EPHEMERIS_TYPE', 'CLASSIFICATION_TYPE', 'NORAD_CAT_ID',
       'ELEMENT_SET_NO', 'REV_AT_EPOCH', 'BSTAR', 'MEAN_MOTION_DOT',
       'MEAN_MOTION_DDOT'],
      dtype='object')

In [8]:
import numpy as np
import pandas as pd


def calculate_orbital_parameters(df):
    # Constants
    GM_EARTH = 398600  # km^3/s^2

    # Calculate semi-major axis from mean motion
    df["a_km"] = (GM_EARTH / (df["MEAN_MOTION"] * (2 * np.pi / 86400) ** 2)) ** (
        1 / 3
    )  # mean motion in rad/s

    # Calculate perihelion distance
    df["q_au"] = df["a_km"] * (1 - df["ECCENTRICITY"])

    # Calculate aphelion distance
    df["Q_au"] = df["a_km"] * (1 + df["ECCENTRICITY"])

    # Calculate orbital period
    df["P_yr"] = (
        2 * np.pi * np.sqrt(df["a_km"] ** 3 / GM_EARTH) / (365.25 * 24 * 3600)
    )  # convert to years

    # Calculate mean anomaly at epoch (assuming M0 = 0 for simplicity)
    df["M"] = df["REV_AT_EPOCH"] * 360 + 0  # Adjust according to initial conditions

    return df

# Calculate missing parameters
df_calculated = calculate_orbital_parameters(df)
df_calculated.head()

,OBJECT_NAME,OBJECT_ID,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,...,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,a_km,q_au,Q_au,P_yr,M
0,CALSPHERE 1,1964-063C,2024-10-04T21:04:06.723840,13.753923,0.002531,90.2077,57.7181,13.2889,140.5397,0,...,999,98610,0.003792,3.659000e-05,0.0,17630.291135,17585.665342,17674.916928,0.000738,35499600
1,CALSPHERE 2,1964-063E,2024-10-04T21:20:37.934880,13.528209,0.001716,90.2215,61.4004,223.8250,307.5836,0,...,999,77239,0.000264,1.840000e-06,0.0,17727.802839,17697.390793,17758.214885,0.000744,27806040
2,LCS 1,1965-034C,2024-10-04T20:44:33.029952,9.893067,0.000814,32.1379,209.8915,78.6346,281.5066,0,...,999,14845,0.002065,2.400000e-07,0.0,19676.961520,19660.948409,19692.974631,0.000870,5344200
3,TEMPSAT 1,1965-065E,2024-10-04T20:36:45.512640,13.335324,0.006918,89.9646,213.6337,189.0694,232.1401,0,...,999,87737,0.000278,1.480000e-06,0.0,17812.867028,17689.637614,17936.096443,0.000750,31585320
4,CALSPHERE 4A,1965-065H,2024-10-04T20:04:34.135680,13.361109,0.007081,89.9358,127.4565,54.7174,75.9291,0,...,999,87986,0.000580,3.110000e-06,0.0,17801.400581,17675.341742,17927.459419,0.000749,31674960


In [10]:
def map_dataframe_to_json_fields(df):
    # Define the mapping from DataFrame columns to JSON keys
    column_mapping = {
        "OBJECT_NAME": "object",
        "EPOCH": "epoch_tdb",
        "MEAN_ANOMALY": "tp_tdb",  # Assuming you want to map to tp_tdb based on context
        "ECCENTRICITY": "e",
        "INCLINATION": "i_deg",
        "ARG_OF_PERICENTER": "w_deg",
        "RA_OF_ASC_NODE": "node_deg",
        "q_au": "q_au_1",  # Assuming q_au is the perihelion distance
        "Q_au": "q_au_2",  # Assuming Q_au is the aphelion distance
        "P_yr": "p_yr",
        "REV_AT_EPOCH": "moid_au",  # Mapping to a placeholder, update if you have the actual MOID
        "BSTAR": "a1_au_d_2",  # Mapping as placeholders, update with actual data
        "MEAN_MOTION_DOT": "a2_au_d_2",
        "OBJECT_ID": "object_name",  # Mapping OBJECT_ID to object_name for consistency
        "MEAN_MOTION": "mean_motion",  # If you want to keep it for reference
        "MEAN_MOTION_DDOT": "mean_motion_ddot",  # If you want to keep it for reference
    }

    # Rename the DataFrame columns based on the mapping
    df_renamed = df.rename(columns=column_mapping)

    return df_renamed
# Apply the mapping function
df_mapped = map_dataframe_to_json_fields(df)
df_mapped.head()

,object,object_name,epoch_tdb,mean_motion,e,i_deg,node_deg,w_deg,tp_tdb,EPHEMERIS_TYPE,...,ELEMENT_SET_NO,moid_au,a1_au_d_2,a2_au_d_2,mean_motion_ddot,a_km,q_au_1,q_au_2,p_yr,M
0,CALSPHERE 1,1964-063C,2024-10-04T21:04:06.723840,13.753923,0.002531,90.2077,57.7181,13.2889,140.5397,0,...,999,98610,0.003792,3.659000e-05,0.0,17630.291135,17585.665342,17674.916928,0.000738,35499600
1,CALSPHERE 2,1964-063E,2024-10-04T21:20:37.934880,13.528209,0.001716,90.2215,61.4004,223.8250,307.5836,0,...,999,77239,0.000264,1.840000e-06,0.0,17727.802839,17697.390793,17758.214885,0.000744,27806040
2,LCS 1,1965-034C,2024-10-04T20:44:33.029952,9.893067,0.000814,32.1379,209.8915,78.6346,281.5066,0,...,999,14845,0.002065,2.400000e-07,0.0,19676.961520,19660.948409,19692.974631,0.000870,5344200
3,TEMPSAT 1,1965-065E,2024-10-04T20:36:45.512640,13.335324,0.006918,89.9646,213.6337,189.0694,232.1401,0,...,999,87737,0.000278,1.480000e-06,0.0,17812.867028,17689.637614,17936.096443,0.000750,31585320
4,CALSPHERE 4A,1965-065H,2024-10-04T20:04:34.135680,13.361109,0.007081,89.9358,127.4565,54.7174,75.9291,0,...,999,87986,0.000580,3.110000e-06,0.0,17801.400581,17675.341742,17927.459419,0.000749,31674960


In [16]:
df_mapped[-100:].to_json("satellites.json", orient="records")

In [14]:
df_mapped

,object,object_name,epoch_tdb,mean_motion,e,i_deg,node_deg,w_deg,tp_tdb,EPHEMERIS_TYPE,...,ELEMENT_SET_NO,moid_au,a1_au_d_2,a2_au_d_2,mean_motion_ddot,a_km,q_au_1,q_au_2,p_yr,M
0,CALSPHERE 1,1964-063C,2024-10-04T21:04:06.723840,13.753923,0.002531,90.2077,57.7181,13.2889,140.5397,0,...,999,98610,0.003792,3.659000e-05,0.0,17630.291135,17585.665342,17674.916928,0.000738,35499600
1,CALSPHERE 2,1964-063E,2024-10-04T21:20:37.934880,13.528209,0.001716,90.2215,61.4004,223.8250,307.5836,0,...,999,77239,0.000264,1.840000e-06,0.0,17727.802839,17697.390793,17758.214885,0.000744,27806040
2,LCS 1,1965-034C,2024-10-04T20:44:33.029952,9.893067,0.000814,32.1379,209.8915,78.6346,281.5066,0,...,999,14845,0.002065,2.400000e-07,0.0,19676.961520,19660.948409,19692.974631,0.000870,5344200
3,TEMPSAT 1,1965-065E,2024-10-04T20:36:45.512640,13.335324,0.006918,89.9646,213.6337,189.0694,232.1401,0,...,999,87737,0.000278,1.480000e-06,0.0,17812.867028,17689.637614,17936.096443,0.000750,31585320
4,CALSPHERE 4A,1965-065H,2024-10-04T20:04:34.135680,13.361109,0.007081,89.9358,127.4565,54.7174,75.9291,0,...,999,87986,0.000580,3.110000e-06,0.0,17801.400581,17675.341742,17927.459419,0.000749,31674960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10285,STARLINK-32447,2024-175U,2024-10-05T06:00:00.999648,15.852138,0.000052,53.1645,103.4736,56.6065,296.1418,0,...,999,279,-0.003938,-8.655200e-03,0.0,16815.343246,16814.475574,16816.210917,0.000688,100440
10286,STARLINK-32446,2024-175V,2024-10-05T06:00:00.999648,15.851788,0.000052,53.1647,103.4810,56.8274,292.3925,0,...,999,280,-0.003874,-8.514870e-03,0.0,16815.466950,16814.587501,16816.346399,0.000688,100800
10287,SHIJIAN-19 (SJ-19),2024-177A,2024-10-04T21:16:18.754752,15.783577,0.000778,41.6002,113.1464,347.2820,12.7830,0,...,999,132,0.000116,2.033200e-04,0.0,16839.655644,16826.547656,16852.763632,0.000689,47520
10288,CREW DRAGON 9,2024-178A,2024-10-04T21:19:59.117952,15.491997,0.000919,51.6362,132.0170,38.0613,322.1023,0,...,999,47550,0.000420,2.297100e-04,0.0,16944.647971,16929.079229,16960.216714,0.000696,17118000
